20221010 

In [48]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
from bokeh.io import export_png
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [49]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,
                  GFP_DNA,GFP_RNA,GFP,
                  Repressor_DNA,Repressor_RNA,Repressor,
                  R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.001
T7_DNA.lc =0.001

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 0.1
GFP_DNA.lc = 0.1

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.5
Repressor_DNA.lc = 0.5


#resources
R.ic =100
R.lc =100

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):

    molecules_0[i] = molecules_list[i].ic


In [50]:
#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [51]:
for conc in [1,0.1,0.01,0.001,0.0001]:
    T7_DNA.ic = conc
    T7_DNA.lc = conc
    #reset molecules_0

    for i in range(0,len(molecules_list)):

        molecules_0[i] = molecules_list[i].ic


    #stage 1 kick start
    result_all=[None]*len(molecules_list) #initial condition

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)

    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)

    #wash out
    dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)

    #load data to object 
    result_all = np.resize(result_all,(96,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]



    t = np.linspace(0, 1440, 96)
    p = bokeh.plotting.figure(
        plot_width=800,
        plot_height=400,
        x_axis_label="t",
        y_axis_type="linear",

    )

    colors = bokeh.palettes.d3["Category10"][3]


    # Populate glyphs
    p.line(
        t/60, GFP.result, line_width=2, color=colors[0]
    )
    vline1 = Span(location=4, dimension='height', line_color='black', line_width=1,line_dash='dashed')
    vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
    p.add_layout(vline1)
    p.add_layout(vline2)
    file_name = "plot_{}.png".format(conc)
    export_png(p, filename=file_name)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58

In [39]:
file_name

'plot_0.0001.png'

In [47]:
for i in range(0,4):
    print(i)

0
1
2
3
